# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import logging

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,274.21,86,75,4.63,CL,Mon Jul 29 00:16:53 2024
1,1,pyapon,16.2833,95.6833,299.72,87,100,1.22,MM,Mon Jul 29 00:16:54 2024
2,2,puerto deseado,-47.7503,-65.8938,275.11,68,26,6.25,AR,Mon Jul 29 00:16:55 2024
3,3,hithadhoo,-0.6000,73.0833,302.09,71,100,4.57,MV,Mon Jul 29 00:16:56 2024
4,4,grytviken,-54.2811,-36.5092,271.99,81,62,4.78,GS,Mon Jul 29 00:08:28 2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
df_narrow = city_data_df.loc[21 < city_data_df['Max Temp']]

# Drop any rows with null values
df_narrow.dropna

# Display sample data
df_narrow.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,274.21,86,75,4.63,CL,Mon Jul 29 00:16:53 2024
1,1,pyapon,16.2833,95.6833,299.72,87,100,1.22,MM,Mon Jul 29 00:16:54 2024
2,2,puerto deseado,-47.7503,-65.8938,275.11,68,26,6.25,AR,Mon Jul 29 00:16:55 2024
3,3,hithadhoo,-0.6000,73.0833,302.09,71,100,4.57,MV,Mon Jul 29 00:16:56 2024
4,4,grytviken,-54.2811,-36.5092,271.99,81,62,4.78,GS,Mon Jul 29 00:08:28 2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df_narrow[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

C:\Users\Moburu\AppData\Local\Temp\ipykernel_26060\1510133662.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,punta arenas,CL,-53.1500,-70.9167,86,
1,pyapon,MM,16.2833,95.6833,87,
2,puerto deseado,AR,-47.7503,-65.8938,68,
3,hithadhoo,MV,-0.6000,73.0833,71,
4,grytviken,GS,-54.2811,-36.5092,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'limit': 1,
    'filter': [],
    'bias': [],
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    print(name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as Argument:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        logging.exception('Error message:')

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hotel Lacolet', 'country': 'Chile', 'country_code': 'cl', 'state': 'Region of Magallanes and Chilean Antarctica', 'county': 'Provincia de Magallanes', 'city': 'Punta Arenas', 'postcode': '6200729', 'district': 'Cerro de la Cruz', 'street': 'Avenida Cristobal Colón', 'lon': -70.9144173, 'lat': -53.1566042, 'formatted': 'Hotel Lacolet, Avenida Cristobal Colón, 620 0729 Punta Arenas, Chile', 'address_line1': 'Hotel Lacolet', 'address_line2': 'Avenida Cristobal Colón, 620 0729 Punta Arenas, Chile', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Hotel Lacolet', 'osm_id': 5351299530, 'tourism': 'hotel', 'osm_type': 'n'}}, 'distance': 751, 'place_id': '51619923d085ba51c

ERROR:root:Error message:
Traceback (most recent call last):
  File "C:\Users\Moburu\AppData\Local\Temp\ipykernel_26060\1195407018.py", line 36, in <module>
    hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
                                        ~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


{'type': 'FeatureCollection', 'features': []}
pyapon - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Los Olmos', 'country': 'Argentina', 'country_code': 'ar', 'state': 'Santa Cruz Province', 'county': 'Deseado', 'city': 'Puerto Deseado', 'postcode': '9050', 'district': 'Quinta Cadario', 'street': 'Gobernador Gregores', 'lon': -65.8909923, 'lat': -47.7522432, 'formatted': 'Los Olmos, Gobernador Gregores, Quinta Cadario, 9050 Puerto Deseado, Argentina', 'address_line1': 'Los Olmos', 'address_line2': 'Gobernador Gregores, Quinta Cadario, 9050 Puerto Deseado, Argentina', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Los Olmos', 'osm_id': 1882650503, 'tourism': 'hotel', 'osm_type': 'n'}}, 'distance': 302

ERROR:root:Error message:
Traceback (most recent call last):
  File "C:\Users\Moburu\AppData\Local\Temp\ipykernel_26060\1195407018.py", line 36, in <module>
    hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
                                        ~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


{'type': 'FeatureCollection', 'features': []}
grytviken - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Kang Lha Chen', 'country': 'India', 'country_code': 'in', 'state': 'Ladakh', 'state_district': 'Leh District', 'county': 'Leh Tehsil', 'city': 'Leh', 'suburb': 'Zangsti', 'street': 'Changspa Road', 'lon': 77.58363618701208, 'lat': 34.1667552, 'formatted': 'Kang Lha Chen, Changspa Road, Zangsti, Leh -, Ladakh, India', 'address_line1': 'Kang Lha Chen', 'address_line2': 'Changspa Road, Zangsti, Leh -, Ladakh, India', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Kang Lha Chen', 'osm_id': 344057387, 'tourism': 'hotel', 'building': 'yes', 'osm_type': 'w'}}, 'place_

...

ERROR:root:Error message:
Traceback (most recent call last):
  File "C:\Users\Moburu\AppData\Local\Temp\ipykernel_26060\1195407018.py", line 36, in <module>
    hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
                                        ~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


{'type': 'FeatureCollection', 'features': []}
wanning - nearest hotel: No hotel found


ERROR:root:Error message:
Traceback (most recent call last):
  File "C:\Users\Moburu\AppData\Local\Temp\ipykernel_26060\1195407018.py", line 36, in <module>
    hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
                                        ~~~~~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


{'type': 'FeatureCollection', 'features': []}
yardimli - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,punta arenas,CL,-53.1500,-70.9167,86,Hotel Lacolet
1,pyapon,MM,16.2833,95.6833,87,No hotel found
2,puerto deseado,AR,-47.7503,-65.8938,68,Los Olmos
3,hithadhoo,MV,-0.6000,73.0833,71,Pebbles Inn
4,grytviken,GS,-54.2811,-36.5092,81,No hotel found
...,...,...,...,...,...,...
573,tobol'sk,RU,58.2000,68.2667,67,Ямская
574,port-vila,VU,-17.7338,168.3219,57,Holiday Inn Resort Vanuatu
575,nabire,ID,-3.3667,135.4833,69,Hotel Anggrek
576,wanning,CN,18.8003,110.3967,85,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    hover = True,
    hover_cols = ['Country', 'Hotel Name']

)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)